In [ ]:
import numpy as np
import torch
import os
import logging
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
import seaborn as sns
import pandas as pd


from nequip.data import AtomicDataDict
from nequip.nn import (
    GraphModuleMixin,
    SequentialGraphNetwork,
    AtomwiseLinear,
    AtomwiseReduce,
    ForceOutput,
    PerSpeciesScaleShift,
    ConvNetLayer,
)
from nequip.nn.embedding import (
    OneHotAtomEncoding,
    RadialBasisEdgeEncoding,
    SphericalHarmonicEdgeAttrs,
)

from nequip.utils import Config
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
from nequip.data import AtomicData

In [ ]:
class Model:
    
    def __init__(self,PathToModel,PathToData,TestSetSize):
        
        self.ParentPath=os.getcwd()
        self.PathToData=os.path.join(self.ParentPath,PathToData)
        self.PathToModel=os.path.join(self.ParentPath,PathToModel)
        self.TestSetSize=TestSetSize
#         self.PathToPrediction=os.path.join(self.PathToData,"ML/")
        
#         if os.path.exists(self.PathToPrediction):
#             shutil.rmtree(self.PathToPrediction)
#         os.mkdir(self.PathToPrediction); print(f"Created {self.PathToPrediction}")
        
    def GetDataAndNumberOfAtoms(self):
        
        
        data=np.load(self.PathToData)
        return data,data['z']
        
        
#         os.chdir(self.PathToData)
#         os.system("vibes u traj 2db trajectory.son")
#         atoms=read(self.PathToData+'trajectory.db',index=':')
#         self.NumberOfAtoms=len(atoms[0].numbers)
#         print(f"Number of atoms = {len(atoms[0].numbers)}")
#         return np.array(atoms[0].numbers)
    
    
    def PredictForcesAndEnergies(self):
        
        torch.cuda.empty_cache()
        print(self.ParentPath)
        print(self.PathToModel)
        config = Config.from_file(self.PathToModel+'tutorial.yaml')
        final_model=torch.load(self.PathToModel+'deployed.pth')
        final_model.eval()
        
        Data,z = self.GetDataAndNumberOfAtoms()
        
        PredictedForcesList=np.empty([len(z),3,0]); PredictedEnergiesList=np.empty([0,1])
        ActualForcesList=np.empty([len(z),3,0]); ActualEnergiesList=np.empty([0,1])
        
        for k in range(self.TestSetSize):
            print(f"Running point {k}")
            i=np.random.randint(1,len(Data['R']))
            r = Data['R'][i] #+3
            forces = Data['F'][i]
            energy=Data['E'][i]
            #energies_list.append(energy)
            
            data = AtomicData.from_points(
        pos=r,
        r_max=config['r_max'],
        **{AtomicDataDict.ATOMIC_NUMBERS_KEY: torch.Tensor(torch.from_numpy(z.astype(np.float32))).to(torch.int64)}
    )
            data = data.to('cpu')

            ForcePrediction = final_model(AtomicData.to_AtomicDataDict(data))['forces'].detach().cpu().numpy()
            EnergyPrediction = final_model(AtomicData.to_AtomicDataDict(data))['total_energy'].detach().cpu().numpy()
            
            
            PredictedForcesList=np.dstack((PredictedForcesList,ForcePrediction))
            PredictedEnergiesList=np.vstack((PredictedEnergiesList,EnergyPrediction))

            ActualForcesList=np.dstack((ActualForcesList,forces))
            ActualEnergiesList=np.vstack((ActualEnergiesList,energy))
            
            
            
        return PredictedForcesList,PredictedEnergiesList.reshape(-1),\
    ActualForcesList,ActualEnergiesList.reshape(-1)
#         return PredictedForcesList.flatten(),\
#     PredictedEnergiesList.flatten(),ActualForcesList.flatten(),ActualEnergiesList.flatten()
    

# Tests

In [ ]:
RSModel=Model(PathToModel='nvt.1500.RS/',PathToData='nvt.1500/tutorial_data/trajectoryTest.npz',
             TestSetSize=2)

PredictedForces,PredictedEnergies,\
ActualForces,ActualEnergies \
=RSModel.PredictForcesAndEnergies()

n=PredictedForces.shape[-1]





MAEOverAvg=np.mean(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)\
/np.mean(np.abs(ActualForces.reshape(-1,n)),axis=0)

MAEs=np.mean(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)

MaxEOverAvg=np.max(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)\
/np.mean(np.abs(ActualForces.reshape(-1,n)),axis=0)

RS=pd.DataFrame({'Energies':ActualEnergies,'MAEs':MAEs,'MAEOverAvg':MAEOverAvg,
                                'MaxEOverAvg':MaxEOverAvg})







In [ ]:
NRSModel=Model(PathToModel='nvt.1500.TRAINEDLONGER/',PathToData='nvt.1500/tutorial_data/trajectoryTest.npz',
             TestSetSize=2)

PredictedForces,PredictedEnergies,\
ActualForces,ActualEnergies \
=RSModel.PredictForcesAndEnergies()

n=PredictedForces.shape[-1]





MAEOverAvg=np.mean(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)\
/np.mean(np.abs(ActualForces.reshape(-1,n)),axis=0)

MAEs=np.mean(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)

MaxEOverAvg=np.max(np.abs(ActualForces.reshape(-1,n))-np.abs(PredictedForces.reshape(-1,n)),axis=0)\
/np.mean(np.abs(ActualForces.reshape(-1,n)),axis=0)

NRS=pd.DataFrame({'Energies':ActualEnergies,'MAEs':MAEs,'MAEOverAvg':MAEOverAvg,
                                'MaxEOverAvg':MaxEOverAvg})






In [ ]:
AvgMAERS=str(np.round(RS["MAEs"].mean(),3))
AvgMAENonRS=str(np.round(NRS["MAEs"].mean(),3))

df=pd.concat([RS,NRS], keys=['RS, Avg MAE (eV/Atom) = '+AvgMAERS,
                                           'N-RS, Avg MAE (eV/Atom) = '+AvgMAENonRS])
df.index = [index[0] for index in df.index.values]
sns.set(font_scale = 1.2)
#plt.figure(figsize=(20,30))
g=sns.jointplot(data=df,x="Energies",y="MAEs",hue=df.index,height=7.5)

#g.ax_joint.set_xscale('log')
g.ax_joint.set_yscale('log')
g.set_axis_labels('Energies (eV)', 'MAEs (eV/Angstrom)', fontsize=16)
g.ax_joint.axhline(y=RS["MAEs"].mean(),linestyle='--',c='blue',label='Avg MAE RS')
g.ax_joint.axhline(y=NRS["MAEs"].mean(),linestyle='--',c='orange',label='Avg MAE NRS')